In [1]:
# Importing libraries
import numpy as np
import tensorflow as tf
import gym
import matplotlib.pyplot as plt

In [2]:
# Network hyperparameters
policy_hidden = 64

# for baseline
value_hidden = 64
value_out = 1

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
# Policy Network
class PolicyNet(tf.keras.Model):
    def __init__(self, num_actions):
        super(PolicyNet, self).__init__()
        self.dense1 = tf.keras.layers.Dense(policy_hidden, activation='relu')
        self.dense2 = tf.keras.layers.Dense(num_actions, activation='softmax')

    def call(self, inp):
        d1 = self.dense1(inp)
        d2 = self.dense2(d1)
        return d2

In [4]:
# Value Network for baseline
class ValueNet(tf.keras.Model):
    def __init__(self):
        super(ValueNet, self).__init__()
        self.dense1 = tf.keras.layers.Dense(value_hidden, activation='relu')
        self.dense2 = tf.keras.layers.Dense(value_out)

    def call(self, inp):
        d1 = self.dense1(inp)
        d2 = self.dense2(d1)
        return d2

In [ ]:
class REINFORCE():
    def __init__(self, baseline):
        # Creating environment
        self.env = gym.make('Acrobot-v1')

        # Environment parameters
        self.num_actions = self.env.action_space.n
        self.state_size = self.env.observation_space.shape[0]

        # Initializing networks and optimizers based on baseline = True or False
        self.baseline = baseline
        if not self.baseline:
            self.policy = PolicyNet(self.num_actions)
            self.optimizer = tf.keras.optimizers.Adam(learning_rate)
        else:
            self.policy = PolicyNet(self.num_actions)
            self.optimizer = tf.keras.optimizers.Adam(learning_rate)
            self.value_net = ValueNet()
            self.val_optimizer = tf.keras.optimizers.Adam(learning_rate)


    def choose_action(self, state):
        # Choosing action using probabilities determined by policy network
        state_re = tf.expand_dims(state, axis=0)
        probs = self.policy(state_re).numpy()[0]
        action = np.random.choice(len(probs), p=probs)
        return action


    def calc_discounted_rew(self, rewards):
        # Computing discounted reward, Gt
        rshape = len(rewards)
        disc_rew = np.zeros((rshape,), dtype=np.float32)
        Gt = 0
        for r in reversed(range(len(rewards))):
            Gt = Gt * gamma + rewards[r]
            disc_rew[r] = Gt
        return disc_rew

    def train_policy_net(self, states, actions, disc_rew):
        # Optimizing policy network without baseline
        with tf.GradientTape() as tape:
            act_probs = self.policy(states)
            act_mask = tf.one_hot(actions, depth=act_probs.shape[1])
            log_probs = tf.reduce_sum(act_mask * tf.math.log(act_probs), axis=1)
            loss = -tf.reduce_mean(log_probs * disc_rew)

        grad = tape.gradient(loss, self.policy.trainable_variables)
        self.optimizer.apply_gradients(zip(grad, self.policy.trainable_variables))
        return loss


    def train_policy_net_baseline(self, states, actions, disc_rew):
        # Optimizing policy network with baseline
        with tf.GradientTape() as tape:
            act_probs = self.policy(states)
            act_mask = tf.one_hot(actions, depth=act_probs.shape[1])
            log_probs = tf.reduce_sum(act_mask * tf.math.log(act_probs), axis=1)
            adv = disc_rew - self.value_net(tf.convert_to_tensor(states, dtype=tf.float32))
            loss = -tf.reduce_mean(log_probs * adv)

        grad = tape.gradient(loss, self.policy.trainable_variables)
        self.optimizer.apply_gradients(zip(grad, self.policy.trainable_variables))
        return loss


    def train_value_net(self, states, disc_rew):
        # Optimizing value network to obtain baseline
        with tf.GradientTape() as tape:
            values = self.value_net(states)
            loss = tf.reduce_mean(tf.square(disc_rew - tf.squeeze(values)))

        grad = tape.gradient(loss, self.value_net.trainable_variables)
        self.val_optimizer.apply_gradients(zip(grad, self.value_net.trainable_variables))


    def train(self):
        episode_rewards_total = []

        for ep in range(num_episodes):
            ep_states = []
            ep_actions = []
            ep_rewards = []

            state = self.env.reset()

            done = False
            while not done:
                action = self.choose_action(state)
                next_state, reward, done, _ = self.env.step(action)

                ep_states.append(state)
                ep_actions.append(action)
                ep_rewards.append(reward)

                state = next_state

            discounted_rewards = self.calc_discounted_rew(ep_rewards)
            states = np.array(ep_states)
            actions = np.array(ep_actions)
            if self.baseline:
                loss = self.train_policy_net_baseline(states, actions, discounted_rewards)
                self.train_value_net(states, discounted_rewards)
            if not self.baseline:
                loss = self.train_policy_net(states, actions, discounted_rewards)


            cumulative_reward = np.sum(ep_rewards)
            episode_rewards_total.append(cumulative_reward)

            print(f"Episode {ep}: Cumulative Reward = {cumulative_reward}")

        self.env.close()
        return episode_rewards_total



In [32]:
# Model hyperparameters
gamma = 0.99
learning_rate = 0.001
num_episodes = 500

In [33]:
# Training REINFORCE models over 5 runs to account for stochasticity
num_runs = 5

In [ ]:
reward_over_runs_wob = []
reward_over_runs_b = []
for r in range(num_runs):
    # Creating model without baseline
    model_wob = REINFORCE(False)
    # Training model without baseline
    rewards_wob = model_wob.train()
    reward_over_runs_wob.append(rewards_wob)

    # Creating model with baseline
    model_b = REINFORCE(True)
    # Training model with baseline
    rewards_b = model_b.train()
    reward_over_runs_b.append(rewards_b)

In [35]:
returns_wob_arr = np.asarray(reward_over_runs_wob)
returns_b_arr = np.asarray(reward_over_runs_b)

# Computing mean over 5 runs across 500 episodes
mean_return_wob = np.mean(returns_wob_arr, axis=0)
mean_return_ba = np.mean(returns_b_arr, axis=0)

# Computing standard deviation over 5 runs across 500 episodes
std_return_wob = np.std(returns_wob_arr, axis=0)
std_return_ba = np.std(returns_b_arr, axis=0)

In [ ]:
# Comparison by plotting
plt.figure()
plt.plot(mean_return_ba, c='r', label='REINFORCE_w_baseline')
plt.fill_between(np.arange(0,num_episodes), mean_return_ba+ std_return_ba, mean_return_ba - std_return_ba, alpha=0.2)
plt.plot(mean_return_wob, c='b', label='REINFORCE_wout_baseline')
plt.fill_between(np.arange(0,num_episodes), mean_return_wob+ std_return_wob, mean_return_wob - std_return_wob, alpha=0.2)
plt.xlabel('Episode Number')
plt.ylabel('Episodic Return')
plt.title(f'REINFORCE (Acrobot v1)')
plt.legend()
plt.show()